<h3> Obtaining DNA Sequences from Various Methods</h3>

In [1]:
from Bio import SeqIO
from Bio import Entrez
Entrez.email = "bryan@gmail.com"

#### Method 1: Simple Text (String)

Creating a nucleotide sequence from a randomly generated string representing the nucleotide bases

In [2]:
import random
#Define a list of the nucleotide bases
Nucleotide = ["G", "T", "A", "C"]

# Use random.choice method from random module to generate concatenated string
# With the given length of 50, you can change it depending on how long the sequence you want to be
DNAString = "".join([random.choice(Nucleotide) for nuc in range(50)])

DNAString

'TATCATTGGCTGACTTGCTCCCACGGCGCTGACTGAAAACAGGTCGGGCG'

#### Method 2 Parsing a URL

In [3]:
#Define the functions
from urllib.parse import urlparse

def extract_accession_number(url):
    segments = urlparse(url).path.split("/")
    accession_number = segments[-1]
    return accession_number

def getnuc_from_ncbi(url):
    accession_number = extract_accession_number(url)
    if not accession_number:
        print("Invalid URL. Please provide a valid NCBI nucleotide sequence URL.")
        return
    handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record

def save_fasta(record):
    with open("output/output_seq.fasta", "w") as f:
        fasta_format = f">{record.id} {record.description}\n{record.seq}"
        f.write(fasta_format)



Example Usage

Let's try with taking the amoA gene of Nitrosomonas europaea (a type of ammonia-oxidizing bacteria responsible for converting ammonia into nitrite in the nitrogen cycle). Commonly studied in biological wastewater treatment system.

In [6]:
ncbi_url = "https://www.ncbi.nlm.nih.gov/nuccore/L08050.1"  

record = getnuc_from_ncbi(ncbi_url)
save_fasta(record)
print(record)

ID: L08050.1
Name: NTOAMOAB
Description: Nitrosomonas europaea ammonia monooxygenase acetylene-binding protein (amoA) gene, complete cds; ammonia monooxygenase (amoB) gene, complete cds
Number of features: 6
/molecule_type=DNA
/topology=linear
/data_file_division=BCT
/date=11-MAY-1995
/accessions=['L08050']
/sequence_version=1
/keywords=['acetylene-binding protein', 'ammonia monooxygenase', 'amoA gene', 'amoB gene']
/source=Nitrosomonas europaea
/organism=Nitrosomonas europaea
/taxonomy=['Bacteria', 'Pseudomonadota', 'Betaproteobacteria', 'Nitrosomonadales', 'Nitrosomonadaceae', 'Nitrosomonas']
/references=[Reference(title='Sequence of the gene coding for ammonia monooxygenase in Nitrosomonas europaea', ...), Reference(title='Sequence of the gene, amoB, for the 43-kDa polypeptide of ammonia monoxygenase of Nitrosomonas europaea', ...), Reference(title='Direct Submission', ...), Reference(title='Direct Submission', ...)]
/comment=On May 11, 1995 this sequence version replaced gi:791182.

#### Method 3: Reading a Text File as CSV File Format containing the Acession ID and the Name of the Species

In [5]:
import csv
from Bio import SeqIO
from Bio import Entrez

fasta_file = {}
ct = 0


with open('input/methanogens.txt') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        print(row)
        with Entrez.efetch(db="nucleotide", rettype="gb", retmode = "text", id = row[0]) as handle:
            for seq_record in SeqIO.parse(handle, "gb"):
                accession_number = seq_record.annotations['accessions'][0]
                header = f"> {accession_number} {row[1]}"
                fasta_file[header] = str(seq_record.seq)
                ct += 1  
      
print("Number of sequences in file:", ct)

# save all sequences in the multi-fasta file "sequences_general.fasta"

with open("output/sequences_general.fasta","w+") as f:
    for header, sequence in fasta_file.items():
        f.write(f"{header}\n{sequence}\n")

['AF313802.1', 'Methanosaeta concilii strain DSM 3671']
['AF414043.1', 'Methanosarcina mazei strain DSM 2053']
['AY970349.1', 'Methanosaeta harundinacea strain 6Ac ']
['AY970348.1', 'Methanosaeta harundinacea strain 8Ac']
['AY672820.1', 'Methanomethylovorans thermophila ']
['AF414045.1', 'Methanocorpusculum parvum strain DSM 3823']
['KC876049.1', 'Methanomethylovorans uponensis strain EK1']
['GU385700.1', 'Methanobrevibacter smithii strain 4R_4_F02']
['KF836868.1', 'Uncultured Methanoculleus sp. clone HB-107']
['KF836855.1', 'Uncultured Methanoculleus sp. clone XJ6-100']
Number of sequences in file: 10


#### Method 4: Reading a Fasta File Containing Multiple Sequences and Saving as a CSV File Format

In [6]:
from Bio import SeqIO
import pandas as pd

with open("output/sequences_general.fasta",'r') as fasta_f:
    id, description, sequence, lengths = [], [], [], []
    for seq_record in SeqIO.parse(fasta_f,'fasta'):
        id.append(str(seq_record.id))
        description.append(str(seq_record.description))
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

In [7]:
df = pd.DataFrame({
    'Accession ID': id,
    'Name': description,
    'Sequence': sequence,
    'Length': lengths
})
df['Name'] = df['Name'].str.join('').str[10:]
df.head(10)

,Accession ID,Name,Sequence,Length
0,AF313802,Methanosaeta concilii strain DSM 3671,GGGAAGCTACATGTCCGGCGGTGTCGGTTTCACCCAGTACGCTACT...,472
1,AF414043,Methanosarcina mazei strain DSM 2053,TACACAGACGACATCCTCGACAACAACACCTACTATGACGTTGACT...,435
2,AY970349,Methanosaeta harundinacea strain 6Ac,ACGTTCATGGCGTAGTTCGGATAGTTCGCGCCTCTCAGCTCGAGGG...,507
3,AY970348,Methanosaeta harundinacea strain 8Ac,ACGTTCATGGCGTAGTTCGGATAGTTCGCGCCTCTCAGCTCGAGGG...,507
4,AY672820,Methanomethylovorans thermophila,TAGGTCCACGCAGCTCAAGAGGCAGACCTTCGTCGGACTGGTAGGA...,739
5,AF414045,Methanocorpusculum parvum strain DSM 3823,TACACGGATAACATCCTTGATGACTTCACCTACTACGGAATGGACT...,438
6,KC876049,Methanomethylovorans uponensis strain EK1,AGCATGTGTGCCGGTGAAGCAGCAGTCGCTGACCTTTCATATGCAG...,706
7,GU385700,Methanobrevibacter smithii strain 4R_4_F02,ATCTTCAAATTTCTGATGGAGAATACGTTAGCTGCACCACATTGAT...,423
8,KF836868,Uncultured Methanoculleus sp. clone HB-107,GGCATACACCGACAACATCCTGGATGAGTTCACATACTACGGTATG...,436
9,KF836855,Uncultured Methanoculleus sp. clone XJ6-100,GGCCTACACCGACAACATCCTCGATGAGTTCACCTACTACGGTATG...,436


In [8]:
df.to_csv("output/methanogens_seq.csv",index=False)

In [36]:
protein ="MTPISHERGAYAAIAKHSI*SMDAQWNHECFYPRAEFDYQEEGLL*QFWQYMLTQNLCRRESS"
conc_prot = list(protein)
print(conc_prot)

['M', 'T', 'P', 'I', 'S', 'H', 'E', 'R', 'G', 'A', 'Y', 'A', 'A', 'I', 'A', 'K', 'H', 'S', 'I', '*', 'S', 'M', 'D', 'A', 'Q', 'W', 'N', 'H', 'E', 'C', 'F', 'Y', 'P', 'R', 'A', 'E', 'F', 'D', 'Y', 'Q', 'E', 'E', 'G', 'L', 'L', '*', 'Q', 'F', 'W', 'Q', 'Y', 'M', 'L', 'T', 'Q', 'N', 'L', 'C', 'R', 'R', 'E', 'S', 'S']


In [37]:
    def proteins_from_rf(aa_seq):
        """Returns a list of all possible proteins based on the amino acid sequence"""
        current_prot = []
        proteins = []
        for aa in aa_seq:
            if aa == "*":  # Stop Codon
                for p in current_prot:
                    proteins.append(p)
                current_prot = []
            else:
                if aa == "M":  # Start Codon
                    current_prot.append("")
                for i in range(len(current_prot)):
                    current_prot[i] += aa
        return proteins

In [43]:
for rf in proteins_from_rf(conc_prot):
    results = []
    results.append(rf)



MDAQWNHECFYPRAEFDYQEEGLL
